In [1]:
!pip install requests

In [2]:
import requests
from urllib import parse
import pandas as pd

In [3]:
### API 요청 (본인의 키 사용)

In [4]:
# API 요청.(본인의 API 키 사용)
def geocode(address, type='ROAD'):
    result = requests.get("http://api.vworld.kr/req/address?service=address"
        + "&request=getcoord" +"&version2.0" + "&crs=epsg:4326"
        + "&address=" + parse.quote(address) + "&type=" + type
        + "&key=EEF0CBC5-92EC-36C9-9333-84F617E823C8")
    return result

In [13]:
#https://api.vworld.kr/req/address?service=address
#    + &request=getcoord&version=2.0&crs=epsg:4326
#        &address=%ED%9A%A8%EB%A0%B9%EB%A1%9C72%EA%B8%B8%2060
#        &refine=true&simple=false&format=xml&type=road&key=[KEY]

In [5]:
CB_file = pd.read_csv('/Users/hoon/Desktop/빅데이터분석개론실습/실습/week13/chap9/CoffeeBean_2024_2.csv',encoding='cp949',  engine='python')

CB_file.head() #작업 내용 확인용 출력

,store,address,phone,address2
0,학동역 DT점,서울시 강남구 학동로 211 1층,02-3444-9973,서울특별시 강남구 학동로 211 1층
1,수서점,서울시 강남구 광평로 280 수서동 724호,02-3412-2326,서울특별시 강남구 광평로 280 수서동 724호
2,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층
3,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층
4,메가박스점,서울 강남구 삼성동 159 코엑스몰 지하2층,02-6002-3320,서울특별시 강남구 삼성동 159 코엑스몰 지하2층


In [6]:
# 경도, 위도 필드 만들기
CB_file['경도'] = 0.0
CB_file['위도'] = 0.0
CB_file

,store,address,phone,address2,경도,위도
0,학동역 DT점,서울시 강남구 학동로 211 1층,02-3444-9973,서울특별시 강남구 학동로 211 1층,0.0,0.0
1,수서점,서울시 강남구 광평로 280 수서동 724호,02-3412-2326,서울특별시 강남구 광평로 280 수서동 724호,0.0,0.0
2,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층,0.0,0.0
3,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층,0.0,0.0
4,메가박스점,서울 강남구 삼성동 159 코엑스몰 지하2층,02-6002-3320,서울특별시 강남구 삼성동 159 코엑스몰 지하2층,0.0,0.0
...,...,...,...,...,...,...
238,인천공항 제2여객터미널 입구점,인천광역시 중구 운서동 2868 인천공항 제2터미널 1층 연번(23-124),032-743-2894,인천광역시 중구 운서동 2868 인천공항 제2터미널 1층 연번(23-124),0.0,0.0
239,부산서면서전로점,"부산광역시 부산진구 서전로 12, 1층",051-819-8708,"부산광역시 부산진구 서전로 12, 1층",0.0,0.0
240,분당차병원본관앞점,경기도 성남시 분당구 야탑로64 DS타워 1층,031-8017-8243,경기도 성남시 분당구 야탑로64 DS타워 1층,0.0,0.0
241,신용산역래미안점,서울시 용산구 한강대로95,02-2074-1049,서울특별시 용산구 한강대로95,0.0,0.0


In [7]:
# 각 주소마다 API 요청
for i in range(len(CB_file)):
    result = None
    coord_x = coord_y = 0.0
    result = geocode(CB_file.at[i, 'address2']).json()
    if result['response']['status'] != 'NOT_FOUND':
        coord_x = result['response']['result']['point']['x']
        coord_y = result['response']['result']['point']['y']
    else:    # 도로명 주소로 못 찾을 시 지번 주소로 찾기
        result = geocode(CB_file.at[i, 'address2'], 'PARCEL').json()
        if result['response']['status'] != 'NOT_FOUND':
            coord_x = result['response']['result']['point']['x']
            coord_y = result['response']['result']['point']['y']
        else: 
            continue
    
    CB_file.at[i, '경도'] = round(float(coord_x), 4)
    CB_file.at[i, '위도'] = round(float(coord_y), 4)



In [8]:
CB_file

,store,address,phone,address2,경도,위도
0,학동역 DT점,서울시 강남구 학동로 211 1층,02-3444-9973,서울특별시 강남구 학동로 211 1층,127.0321,37.5147
1,수서점,서울시 강남구 광평로 280 수서동 724호,02-3412-2326,서울특별시 강남구 광평로 280 수서동 724호,127.1031,37.4873
2,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층,127.0345,37.5068
3,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층,127.0282,37.4957
4,메가박스점,서울 강남구 삼성동 159 코엑스몰 지하2층,02-6002-3320,서울특별시 강남구 삼성동 159 코엑스몰 지하2층,127.0588,37.5126
...,...,...,...,...,...,...
238,인천공항 제2여객터미널 입구점,인천광역시 중구 운서동 2868 인천공항 제2터미널 1층 연번(23-124),032-743-2894,인천광역시 중구 운서동 2868 인천공항 제2터미널 1층 연번(23-124),126.4338,37.4687
239,부산서면서전로점,"부산광역시 부산진구 서전로 12, 1층",051-819-8708,"부산광역시 부산진구 서전로 12, 1층",129.0609,35.1575
240,분당차병원본관앞점,경기도 성남시 분당구 야탑로64 DS타워 1층,031-8017-8243,경기도 성남시 분당구 야탑로64 DS타워 1층,0.0000,0.0000
241,신용산역래미안점,서울시 용산구 한강대로95,02-2074-1049,서울특별시 용산구 한강대로95,126.9666,37.5288


In [9]:
CB_file.to_csv('/Users/hoon/Desktop/빅데이터분석개론실습/실습/week13/chap9/CB_geoAPI.shp_2024_2.csv',encoding='CP949', index = False)